In [1]:
from mne.filter import filter_data
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pyedflib
import numpy as np
from datetime import timedelta
from scipy.signal import find_peaks_cwt
from scipy.stats import spearmanr

import seaborn as sns

from os import listdir
    #method listdir() returns a list containing the names of the entries in the directory given by path.
from os.path import isfile, join
    #returns true if file in path is an existing regular file

In [2]:
## !! set correct working directory
import os
path="/Users/jeroenhabets/Documents/pilot1_workdata"
os.chdir(path)
os.getcwd() #check

'/Users/jeroenhabets/Documents/pilot1_workdata'

In [3]:
#  read in continuous beeps
esm = df = pd.read_stata(path+'/PRDB_20181122T112857/SANPAR_BE.dta',convert_categoricals = False)
# define the relevant parameters
esm = esm[['subjno', 'mood_well', 'mood_down', 'mood_fright', 'mood_tense', 'phy_sleepy', 'phy_tired',
       'mood_cheerf', 'mood_relax', 'thou_concent', 'pat_hallu', 'loc_where',
       'soc_who', 'soc_who02', 'soc_who03', 'act_what', 'act_what02',
       'act_what03', 'act_norpob', 'sanpar_been', 'sanpar_stil',
       'sanpar_spreken', 'sanpar_lopen', 'sanpar_tremor', 'sanpar_traag',
       'sanpar_stijf', 'sanpar_spann', 'sanpar_beweeg', 'sanpar_onoff',
       'sanpar_medic', 'beep_disturb', '_datetime', '_datetime_e', 'dayno_n', 'beepno_n']]
esm['duration'] = esm['_datetime_e']-esm['_datetime']

mapNames={}
for i in range(20):
    mapNames[9009989+i]=110001+i
### PAY ATTENTION WITH DROP OUT CASTOR 110012 !!!

esm['castorID'] = [mapNames[e] for e in esm['subjno']]

esm['dateOnly'] = [pd.Timestamp.date(esm['_datetime'][row]) for row in range(len(esm))]


# PM: include castor data

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

In [4]:
#  read in morning beeps
morning = df = pd.read_stata(path+'/PRDB_20181122T112857/SANPAR_MOR.dta',convert_categoricals = False)
# define the relevant parameters
morning = morning[['subjno', 'mor_sleptwell', 'sanpar_awake', 'mor_rested', 'sanpar_lich', 
                   'sanpar_tired', '_datetime', '_datetime_e']]
morning['duration'] = morning['_datetime_e']- morning['_datetime']

mapNames={}
for i in range(20):
    mapNames[9009989+i]=110001+i

morning['castorID'] = [mapNames[e] for e in morning['subjno']]

# create dat only item for joining dataframes

morning['dateOnly'] = [pd.Timestamp.date(morning['_datetime'][row]) for row in range(len(morning))]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [5]:
#  read in evening beeps
eve = df = pd.read_stata(path+'/PRDB_20181122T112857/SANPAR_EVE.dta',convert_categoricals = False)
# define the relevant parameters
eve = eve[['subjno', 'sanpar_off','sanpar_lang', 'sanpar_lopen', 'sanpar_kleden', 'sanpar_eten', 'sanpar_hygiene', 'sanpar_huish', 'sanpar_vermoeid', '_datetime', '_datetime_e']]
eve['duration'] = eve['_datetime_e']- eve['_datetime']

mapNames={}
for i in range(20):
    mapNames[9009989+i]=110001+i

eve['castorID'] = [mapNames[e] for e in eve['subjno']]

# create dat only item for joining dataframes
eve['dateOnly'] = [pd.Timestamp.date(eve['_datetime'][row]) for row in range(len(eve))]
# correct for eve's completed after midnight
for row in range(len(eve)):
    if eve['_datetime'][row].hour < 5:
        eve['dateOnly'][row] = eve['dateOnly'][row] - pd.Timedelta('1 day')


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [6]:
# create correct validDays for all valid patients; check code to improve with loops or dictionaries

esm['validDay'] = False 
morning['validDay'] = False 
eve['validDay'] = False 
starts = {110001 : pd.Timestamp(2018,8,28), 110002 : pd.Timestamp(2018,8,29),110003 :pd.Timestamp(2018,8,29),110004 :pd.Timestamp(2018,9,21),110005 :pd.Timestamp(2018,10,2),110006 :pd.Timestamp(2018,10,2),110007 :pd.Timestamp(2018,10,10),110008 :pd.Timestamp(2018,10,19),110009 :pd.Timestamp(2018,10,19),110010 :pd.Timestamp(2018,11,7),110011 :pd.Timestamp(2018,11,8),110012 :pd.Timestamp(2018,11,15)}

listDays ={}
for s in starts.keys():
    listDays[s] = pd.date_range(starts[s],periods=14)

for row in range(len(esm)):
    if esm['dateOnly'].iloc[row] in listDays[esm['castorID'].iloc[row]]:
        esm['validDay'].iloc[row] = True

for row in range(len(morning)):
    if morning['dateOnly'].iloc[row] in listDays[morning['castorID'].iloc[row]]:
        morning['validDay'].iloc[row] = True
for row in range(len(eve)):
    if eve['dateOnly'].iloc[row] in listDays[eve['castorID'].iloc[row]]:
        eve['validDay'].iloc[row] = True

# create dataset with only beeps of valid days
esmValid = esm[esm['validDay'] == True ]
esmValid = esmValid[esmValid['castorID'] != 110012 ]
# create dataset with only beeps of valid days
morning = morning[morning['validDay'] == True ]
morning = morning[morning['castorID'] != 110012 ]
# create dataset with only beeps of valid days
eve = eve[eve['validDay'] == True ]
eve = eve[eve['castorID'] != 110012 ]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
## PRE-PROCESSING, obtaining new-relevant variables

# creating mean columns for positive and negative affect

esmValid['meanPosAff'] = (esmValid['mood_well']+esmValid['mood_cheerf']+esmValid['mood_relax'])/3
esmValid['meanNegAff'] = (esmValid['mood_down']+esmValid['mood_fright']+esmValid['mood_tense'])/3

# create new OnOff parameter with 1 = OFF, 2 = in between, 3 = ON
# 1 = OFF, 3 = ON, 2-4= in between
esmValid['OnOffnew'] = esmValid['sanpar_onoff'][:]
esmValid['OnOffnew'][esmValid['OnOffnew'] == 4] = 2
## 'vectorized programming'

# average stijfheid-traagheid
esmValid['newBrady'] = (esmValid['sanpar_stijf']+esmValid['sanpar_traag'])/2

# add variable on presence of motor fluctuations
esmValid['flucPresence'] = 0 # standard for everybody 0 -> not present
listIDsFluct = [110002, 110003, 110004, 110006]
for id in listIDsFluct:
    esmValid['flucPresence'][esmValid['castorID'] == id] = 1

esmValid['phenotype'] = 0 # standard for everybody 0 -> bradykinetic/rigid type
listIDsTremorDom = [110002, 110007, 110008, 110010, 110011]
for id in listIDsTremorDom:
    esmValid['phenotype'][esmValid['castorID'] == id] = 1

esmValid['DBS'] = 0 # standard for everybody 0 -> no, 1 = yes
listIDsDBS = [110002, 110004, 110006] # 110013 also
for id in listIDsDBS:
    esmValid['DBS'][esmValid['castorID'] == id] = 1

eve['averageFunct_eve'] = (eve['sanpar_lopen']+eve['sanpar_kleden']+eve['sanpar_eten']+
                                eve['sanpar_hygiene']+eve['sanpar_huish'])/5


# creating sub datasets for phenotypes
subTremor = esmValid[esmValid['phenotype'] == 1]
subRigid = esmValid[esmValid['phenotype'] == 0]

# creating sub datasets for presence fluctuations
subFluct = esmValid[esmValid['flucPresence'] == 1]
subNoFluc = esmValid[esmValid['flucPresence'] == 0]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is tryi

In [9]:
# merge continuous, morning and evening dataframes
esmFused = pd.merge(esmValid, morning, how='left', left_on=['castorID','dateOnly'],right_on=['castorID','dateOnly'], suffixes=('', '_MOR')) # adding rsuffix to give extra string to overlapping column names

esmFused = pd.merge(esmFused, eve, how='left', left_on=['castorID','dateOnly'],right_on=['castorID','dateOnly'], suffixes=('', '_EVE')) # adding rsuffix to give extra string to overlapping column names

print(esmValid.shape)
print(esmFused.shape) # check if there are no rows lost or too many new rows after merging

(818, 46)
(821, 69)


In [12]:
## handling missings in ESM rows (1)

checkNanData = esmFused.filter(['mood_well', 'mood_down', 'mood_fright', 'mood_tense',
       'phy_sleepy', 'phy_tired', 'mood_cheerf', 'mood_relax', 'thou_concent',
       'pat_hallu', 'act_norpob', 'sanpar_been',
       'sanpar_stil', 'sanpar_spreken', 'sanpar_lopen', 'sanpar_tremor',
       'sanpar_traag', 'sanpar_stijf', 'sanpar_spann', 'sanpar_beweeg',
       'sanpar_onoff', 'castorID'], axis=1)


# visualize which rows have missings in relevant filtered items
for row in range(len(checkNanData)):
    if checkNanData.iloc[row].isnull().any() == True:
        print(checkNanData.iloc[row])

In [11]:
# performed all three lines
esmFused['sanpar_onoff'][708] = 3
esmFused = esmFused.drop([695, 771]) 


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
esmFused.keys()

Index(['subjno', 'mood_well', 'mood_down', 'mood_fright', 'mood_tense',
       'phy_sleepy', 'phy_tired', 'mood_cheerf', 'mood_relax', 'thou_concent',
       'pat_hallu', 'loc_where', 'soc_who', 'soc_who02', 'soc_who03',
       'act_what', 'act_what02', 'act_what03', 'act_norpob', 'sanpar_been',
       'sanpar_stil', 'sanpar_spreken', 'sanpar_lopen', 'sanpar_tremor',
       'sanpar_traag', 'sanpar_stijf', 'sanpar_spann', 'sanpar_beweeg',
       'sanpar_onoff', 'sanpar_medic', 'beep_disturb', '_datetime',
       '_datetime_e', 'dayno_n', 'beepno_n', 'duration', 'castorID',
       'dateOnly', 'validDay', 'meanPosAff', 'meanNegAff', 'OnOffnew',
       'newBrady', 'flucPresence', 'phenotype', 'DBS', 'subjno_MOR',
       'mor_sleptwell', 'sanpar_awake', 'mor_rested', 'sanpar_lich',
       'sanpar_tired', '_datetime_MOR', '_datetime_e_MOR', 'duration_MOR',
       'validDay_MOR', 'subjno_EVE', 'sanpar_off', 'sanpar_lang',
       'sanpar_lopen_EVE', 'sanpar_kleden', 'sanpar_eten', 'sanpar_hygi

In [14]:
# rename to english
esmFused = esmFused.rename(index=str,columns={'act_norpob':'act_problemless','sanpar_been':'mobility_well',
'sanpar_stil':'sit_still','sanpar_spreken':'speech_well','sanpar_lopen':'walk_well',
'sanpar_tremor':'tremor','sanpar_traag':'slowness','sanpar_stijf':'stiffness',
'sanpar_spann':'muscle_tension','sanpar_beweeg':'dyskinesia','sanpar_awake':'often_awake', 
'sanpar_lich':'tired_phys', 'sanpar_tired':'tired_ment', 'sanpar_off': 'many_offs',
'sanpar_lang':'long_offs', 'sanpar_lopen_EVE':'walk_well_eve', 'sanpar_kleden':'clothing_eve',
'sanpar_eten':'eat_well_eve', 'sanpar_hygiene':'personalcare_eve','sanpar_huish':'household_eve',
'sanpar_vermoeid':'tired_eve'})



In [15]:
## create cell to write relevant dataframe to csv

esmFused.to_csv('prepDataESM11.csv',index=False)